In [1]:
import numpy as np
from sympy import *
from math import floor
import time
from itertools import product
from itertools import combinations
import multiprocessing as mp

In [2]:
def make_linearEqns(A,z):
    #Takes matrix A and variable list z and converts it into a list of linear equations
    z = Matrix(z)
    A = Matrix(A)
    A = A.rref()[0]
    lin_fcns = A*z
    lin_fcns = [f for f in lin_fcns if f != 0]
    return(list(lin_fcns))
#Hypercube functions
def apply_automorphism(R):
    #Applies automorphism so that r1 = 0^k (Not useful for non-hypercubes)
    position = [i for i,r in enumerate(R[0]) if r == '0']
    R_new = [r for r in R]
    for j,r in enumerate(R_new):
        temp = list(r)
        for i in position:
            if temp[i] == '1':
                temp[i] = '0'
            else:
                temp[i] = '1'
        R_new[j] = ''.join(temp)
    return(R_new)

def hypercube_matrix(R):
    A = np.zeros((len(R),len(R[0])))
    for i,r in enumerate(R):
        for j,letter in enumerate(r):
            A[i,j] = int(letter)
    return(A)

def hypercube_polys(k):
    variable_string = ''
    for i in range(1,k+1):
        variable_string = variable_string +'z{},'.format(i)
    variable_string = variable_string[:-1]
    z = var(variable_string)
    P = []
    f = 0
    for i in range(k):
        P.append(z[i]*(z[i]-1)*(z[i]+1))
        f = f + z[i]**2
    fs = []
    for i in range(floor(k/2)):
        fi = f-2*(i+1)
        fs.append(fi)
    return(P,fs,z)

def check_hcube_resolving(A,k):
    P,fs,z = hypercube_polys(k)
    #Get linear functions from A matrix
    lin_fcns = make_linearEqns(A,z)
    #Get Grobner basis of P and linear functions
    G = groebner(P+lin_fcns,order = 'lex')
    for i,fi in enumerate(fs):
        #Compute Grobner basis of G+fi
        Gi = groebner(list(G)+[fi],order = 'lex')
        #Solutions iff Gi neq 1, if Gi neq 1 then R is not resolving
        if not (list(Gi) == [1]):
            return False
    return True
    

In [3]:
def preprocess_hcube(k):
    P,fs,z = hypercube_polys(k)
    Gs = []
    for i,fi in enumerate(fs):
        G = groebner(P+[fi], order = 'lex')
        Gs.append(G)
    return (Gs,z)

In [4]:
def threaded_hcube_resolving(Gs, lin_fcns):
    for G in Gs:
        Gi = groebner(list(G)+lin_fcns, order = 'lex')
        if (list(Gi) != [1]):
            return False
    return True

In [5]:
def generate_ResSet_threaded(Gs,z,k):
    print(k)
    ones_string = '1'*k
    nodes = [''.join(x) for x in product('01',repeat = k)]
    nodes.remove(ones_string)
    random_indices = np.random.permutation(len(nodes))
    R = [ones_string]
    rank = 0
    for i in random_indices:
        node = nodes[i]
        A = hypercube_matrix(R+[node])
        new_rank = np.linalg.matrix_rank(A)
        if new_rank > rank:
            rank = new_rank
            R.append(node)
            lin_fcns = make_linearEqns(A,z)
            is_resolving = threaded_hcube_resolving(Gs,lin_fcns)
            if is_resolving:
                return(R)

In [ ]:
k = 3
num_sets = 4
num_cores = mp.cpu_count()
Sets = []
Gs,z = preprocess_hcube(k)
args = ((Gs,z,k),)*num_sets
with mp.Pool(processes=num_cores) as pool:
    result = pool.map(generate_ResSet_threaded,args)
    print(result.get(timeout = 1))
    

In [ ]:
ks = np.arange(3,7,1)
num_sets = 10
num_cores = mp.cpu_count()
Sets = []
sizes = []
for k in ks:
    Gs,z = preprocess_hcube(k)
    pool = ThreadPool(processes = num_cores)
    args = ((Gs,z,k),)*num_sets
    async_result = [pool.apply_async(generate_ResSet_threaded, args = arg)
                for arg in args]
    return_val = [result.get() for result in async_result]
    Size_list = [len(R) for R in return_val]
    Size_list = set(Size_list)
    sizes.append(list(Size_list))
    Sets.append()